# Decomposing Representations with `infer_change_of_basis`

**6.7970/8.750 Symmetry and its Application to Machine Learning**

This notebook demonstrates how Schur's Lemma and `linalg.infer_change_of_basis` can be used to test whether a representation is irreducible and to identify which irreps live inside a reducible representation.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/characters_cob_demo.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep

## Reference: $P(3)$ irreps and permutation representation

In [ ]:
# Build P(3) from 3D permutation matrices
perm_rep = groups.permutation_matrices(3)  # shape [6, 3, 3]
table = groups.make_multiplication_table(perm_rep)

# Find all irreps
irreps = rep.infer_irreps(table)

# Label them
names = []
for ir in irreps:
    d = ir.shape[1]
    if d == 2:
        names.append("\u0393\u2082 (E)")
    elif d == 1:
        # Check if it's trivial or sign rep
        if np.allclose(ir[:, 0, 0], 1):
            names.append("\u0393\u2081 (A\u2081)")
        else:
            names.append("\u0393\u2081' (A\u2082)")

print("Irreps of P(3):")
for name, ir in zip(names, irreps):
    print(f"  {name}: dim {ir.shape[1]}")

---
## Schur's Lemma: Irreps have a 1D commutant

**Schur's Lemma (Part 1)**: If $D(g) Q = Q D(g)$ for all $g$ and $D$ is an **irrep**, then $Q = c \, \mathbb{I}$.

This means `infer_change_of_basis(irrep, irrep)` should return exactly **1 solution** (a scalar multiple of the identity).

In [ ]:
# Test each irrep: how many Q satisfy D(g) Q = Q D(g)?
for name, ir in zip(names, irreps):
    Q = linalg.infer_change_of_basis(ir, ir)
    print(f"{name}: {len(Q)} solution(s)")
    assert len(Q) == 1, f"Expected 1 solution for irrep {name}!"

print("\nAll irreps have exactly 1 solution \u2014 confirming Schur's Lemma!")

---
## The permutation representation is reducible

If a representation is **reducible**, the commutant space is larger than 1D.

In [ ]:
# How many Q satisfy perm_rep @ Q = Q @ perm_rep?
Q_perm = linalg.infer_change_of_basis(perm_rep, perm_rep)
print(f"Perm rep with itself: {len(Q_perm)} solutions")
print("More than 1 \u21d2 the perm rep is reducible!")

---
## Which irreps are inside the perm rep?

We can test whether each irrep $\Gamma_j$ is contained in the perm rep by checking if there exists a non-trivial $Q$ such that:

$$D^{\text{perm}}(g) \, Q = Q \, D^{(\Gamma_j)}(g) \quad \forall \, g$$

If solutions exist, $\Gamma_j$ is inside the perm rep. The **number of solutions** equals the multiplicity $a_j$.

In [ ]:
# Check each irrep against the perm rep
print("Decomposing the 3D perm rep of P(3):\n")
for name, ir in zip(names, irreps):
    Q = linalg.infer_change_of_basis(perm_rep, ir)
    n = len(Q)
    status = f"\u2713 (appears {n} time{'s' if n != 1 else ''})" if n > 0 else "\u2717 (not contained)"
    print(f"  perm rep \u2283 {name}?  {n} solution(s)  {status}")

print("\n\u21d2  perm rep = \u0393\u2081 \u2295 \u0393\u2082")
print("   Matches the character table decomposition!")

---
## Coming up: the regular representation

We just showed we can identify irreps inside a representation using `infer_change_of_basis`.

But what if we don't know the irreps yet? There is a special representation called the **regular representation** that contains **every** irrep of the group. By decomposing it, we can discover all irreps from scratch.

We will learn how to do this in the [Decomposition & Products notes](https://symm4ml.mit.edu/notes/decomposition-and-products).